# Welcome to TapeAgents!


**TapeAgents** is a framework that leverages a structured, replayable log (**Tape**) of the agent session to facilitate all stages of the LLM Agent development lifecycle. In TapeAgents, the agent reasons by processing the tape and the LLM output to produce new thoughts, actions, control flow steps and append them to the tape. The environment then reacts to the agent’s actions by likewise appending observation steps to the tape.

In [ ]:
from tapeagents.llms import OpenrouterLLM
from dotenv import load_dotenv
import os
load_dotenv()

open_router_api_key = os.environ["OPEN_ROUTER_API_KEY"]
llm = OpenrouterLLM(
    model_name="meta-llama/llama-3.3-70b-instruct:free",  # https://openrouter.ai/meta-llama/llama-3.3-70b-instruct:free
    api_token=open_router_api_key,
    parameters={"temperature": 0.1},
)

/home/dieppham/personal/hack4her/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
llm.quick_response("who is Donald Trump?")

'Donald Trump is the 45th President of the United States, serving from 2017 to 2021. He is a businessman, real estate developer, and television personality who was born on June 14, 1946, in Queens, New York.\n\nBefore entering politics, Trump made a name for himself in the business world, building a real estate empire and creating a brand that became synonymous with luxury and success. He developed numerous high-end properties, including hotels, casinos, and golf courses, and wrote several bestselling books on business and entrepreneurship.\n\nTrump\'s rise to fame also included a successful reality TV show, "The Apprentice," which aired from 2004 to 2015 and featured him as the host and executive producer. He became known for his charismatic personality, his catchphrase "You\'re fired!," and his ability to build a brand and create a sense of excitement around his business ventures.\n\nIn 2015, Trump announced his candidacy for the Republican presidential nomination, running on a platf

## Create environment with the set of tools that the agent can use

In [ ]:
import os

from tapeagents.environment import ToolCollectionEnvironment
from tapeagents.tools.calculator import Calculator
from tapeagents.tools.simple_browser import SimpleBrowser
from tapeagents.tools.web_search import WebSearch

# key for the web search api, you can get a free key at https://serper.dev/
os.environ["SERPER_API_KEY"] = "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"

environment = ToolCollectionEnvironment(tools=[WebSearch(), SimpleBrowser(), Calculator()])
environment.initialize()
print(environment.tools_description())

- WebSearch - Performs a search in the web, wikipedia or youtube
- SimpleBrowser - Simple text-based web browser for reading documents and web pages. Can load web page or document from a URL or file path and scroll through its content.
- Calculator - Tool to evaluate math expressions


## Simple agent that think once and makes one tool call before anwering

In [5]:
from tapeagents.agent import Agent
from tapeagents.nodes import StandardNode, Stop

system_prompt = "You are an assistant that can use web search, browser and calculator. Be brief and concise."
agent = Agent.create(
    llms=llm,
    nodes=[
        # nodes are steps that are executed in order,
        # each node can asks llm to produce a response or call some tool from the environment,
        # when all nodes are executed, the agent will stop
        StandardNode(
            name="think",
            system_prompt=system_prompt,
            guidance="let's think how to use available tools to answer the user's question, do not answer the question yet",
        ),
        StandardNode(
            name="act",
            system_prompt=system_prompt,
            guidance="call the proposed tool",
            use_known_actions=True,  # this will use all the actions defined in the environment
            use_function_calls=True,  # this will allow to use model tool calling capabilities
        ),
        StandardNode(
            name="answer",
            system_prompt=system_prompt,
            guidance="give a final answer to the user's question based on the tool result",
        ),
        Stop(),
    ],
    known_actions=environment.actions(),
    tools_description=environment.tools_description(),
)

In [ ]:
from tapeagents.dialog_tape import DialogTape, UserStep
from tapeagents.orchestrator import main_loop

user_question = "What is the closest distance from the Earth to the Moon?"
tape = DialogTape(steps=[UserStep(content=user_question)])

for event in main_loop(agent, tape, environment):
    if event.agent_event and event.agent_event.step:
        step = event.agent_event.step
        print(f"Agent step ({step.metadata.node}):\n{step.llm_view()}\n---")
    elif event.observation:
        print(f"Tool call result:\n{event.observation.short_view()}\n---")
    elif event.agent_tape:
        tape = event.agent_tape

print("Agent final answer:", tape[-2].reasoning)  # last step before Stop node

Agent step (think):
To find the closest distance from the Earth to the Moon, we can:

1. Use a web search to find information on the Moon's orbit and its closest point to Earth.
2. Utilize a browser to access online resources, such as NASA's website, that provide data on the Earth-Moon distance.
3. Possibly use a calculator to perform calculations if we need to convert units or calculate the distance based on given data.

Let's proceed with the web search to gather more information.
---
Agent step (act):
{
  "kind": "search_action",
  "source": "web",
  "query": "closest distance from Earth to Moon",
  "time_interval": ""
}
---


Search response: {'message': 'Unauthorized.', 'statusCode': 403}
Empty search results, retrying in 2 seconds
Search response: {'message': 'Unauthorized.', 'statusCode': 403}
Empty search results, retrying in 2 seconds
Search response: {'message': 'Unauthorized.', 'statusCode': 403}
Empty search results, retrying in 2 seconds
Failed to search the web: Failed to get search results, try to use browser to access the search engine instead
Traceback (most recent call last):
  File "/home/dieppham/personal/hack4her/tapeagents/tools/web_search.py", line 148, in execute_action
    results = web_search(query)
  File "/home/dieppham/personal/hack4her/tapeagents/tools/web_search.py", line 55, in web_search
    raise Exception("Failed to get search results, try to use browser to access the search engine instead")
Exception: Failed to get search results, try to use browser to access the search engine instead
Action failure: [Errno 2] No such file or directory: '.cache/tool_cache.WebSearch.25695.2569

Tool call result:
{
  "kind": "action_execution_failure",
  "error": "[Errno 2] No such file or directory: '.cache/tool_cache.WebSearch.25695.25695.jsonl'"
}
---
Agent step (answer):
The closest distance from the Earth to the Moon is approximately 225,622 miles (363,104 kilometers). This occurs when the Moon is at its perigee, which is the point in its orbit when it is closest to Earth.
---
Agent step (Stop):
{
  "kind": "final_step",
  "reason": ""
}
---
Agent final answer: The closest distance from the Earth to the Moon is approximately 225,622 miles (363,104 kilometers). This occurs when the Moon is at its perigee, which is the point in its orbit when it is closest to Earth.


## Agent that can think and act in a loop

In [8]:
from tapeagents.core import FinalStep

react_agent = Agent.create(
    llms=llm,
    nodes=[
        StandardNode(
            name="plan",
            system_prompt=system_prompt,
            guidance="write a concise multi-step plan to find the answer for the given question. Do not answer the question yet",
        ),
        StandardNode(
            name="reflect",
            system_prompt=system_prompt,
            guidance="evaluate current state of the solution, propose the next tool to call to move forward. If the plan is finished, call FinalStep to answer the question",
        ),
        StandardNode(
            name="act",
            system_prompt=system_prompt,
            guidance="produce proposed tool call",
            use_known_actions=True,
            use_function_calls=True,
            steps=FinalStep,  # if this step is produced, the agent will stop
            next_node="reflect",  # here we make a loop back to the reflect node
        ),
    ],
    known_actions=environment.actions(),
    tools_description=environment.tools_description(),
)

In [9]:
# question from the GAIA benchmark validation set https://huggingface.co/datasets/gaia-benchmark/GAIA
user_question2 = "If Eliud Kipchoge could maintain his record-making marathon pace indefinitely, how many hours would it take him to run the distance between the Earth and the Moon its closest approach?"
tape = DialogTape(steps=[UserStep(content=user_question2)])

for event in main_loop(react_agent, tape, environment):
    if event.agent_event and event.agent_event.step:
        step = event.agent_event.step
        print(f"Agent step ({step.metadata.node}):\n{step.llm_view()}\n---")
    elif event.observation:
        print(f"Tool call result:\n{event.observation.short_view()}\n---")
    elif event.agent_tape:
        tape = event.agent_tape

print("\n\nAgent final answer:", tape[-1].reason)

Agent step (plan):
To find the answer, follow these steps:

1. Find Eliud Kipchoge's record-making marathon pace (distance per hour).
2. Determine the average distance from the Earth to the Moon at its closest approach.
3. Calculate the total time it would take to cover this distance at Kipchoge's pace by dividing the distance by his speed.
4. Convert the result to hours.
---
Agent step (reflect):
Current state: We have a plan to solve the problem, but we don't have the necessary values for Eliud Kipchoge's record-making marathon pace and the average distance from the Earth to the Moon at its closest approach.

Next tool to call: Web search to find the values for Eliud Kipchoge's record-making marathon pace and the average distance from the Earth to the Moon at its closest approach.
---


Search response: {'message': 'Unauthorized.', 'statusCode': 403}
Empty search results, retrying in 2 seconds


Agent step (act):
{
  "kind": "search_action",
  "source": "web",
  "query": "Eliud Kipchoge marathon pace",
  "time_interval": ""
}
---
Agent step (act):
{
  "kind": "search_action",
  "source": "web",
  "query": "average distance from Earth to Moon at closest approach",
  "time_interval": ""
}
---
Agent step (act):
{
  "kind": "set_next_node",
  "next_node": "reflect"
}
---


Search response: {'message': 'Unauthorized.', 'statusCode': 403}
Empty search results, retrying in 2 seconds
Search response: {'message': 'Unauthorized.', 'statusCode': 403}
Empty search results, retrying in 2 seconds
Failed to search the web: Failed to get search results, try to use browser to access the search engine instead
Traceback (most recent call last):
  File "/home/dieppham/personal/hack4her/tapeagents/tools/web_search.py", line 148, in execute_action
    results = web_search(query)
  File "/home/dieppham/personal/hack4her/tapeagents/tools/web_search.py", line 55, in web_search
    raise Exception("Failed to get search results, try to use browser to access the search engine instead")
Exception: Failed to get search results, try to use browser to access the search engine instead
Action failure: [Errno 2] No such file or directory: '.cache/tool_cache.WebSearch.25695.25695.jsonl'
Traceback (most recent call last):
  File "/home/dieppham/personal/hack4her/tapeagents/tools/base.py

Tool call result:
{
  "kind": "action_execution_failure",
  "error": "[Errno 2] No such file or directory: '.cache/tool_cache.WebSearch.25695.25695.jsonl'"
}
---
Tool call result:
{
  "kind": "action_execution_failure",
  "error": "[Errno 2] No such file or directory: '.cache/tool_cache.WebSearch.25695.25695.jsonl'"
}
---
Agent step (reflect):
Current state: The web search tool failed to execute due to a file not found error. We still need to find Eliud Kipchoge's record-making marathon pace and the average distance from the Earth to the Moon at its closest approach.

Next tool to call: Browser to search for the necessary information.
---
Agent step (act):
{
  "kind": "search_action",
  "source": "web",
  "query": "Eliud Kipchoge marathon pace",
  "time_interval": ""
}
---
Agent step (act):
{
  "kind": "search_action",
  "source": "web",
  "query": "average distance from Earth to Moon at closest approach",
  "time_interval": ""
}
---
Agent step (act):
{
  "kind": "set_next_node",
  "ne